In [1]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pathlib import Path

data_dir = Path("./data")
data_fpath = data_dir / "2022-07-bmvc-report-01-data00-scheduler-params.csv"

import pandas as pd
df = pd.read_csv(data_fpath) 

def check_df(df):
    print(df.shape)
    print(df.dtypes)
    return df.head(5)

check_df(df)

(610, 12)
Name                                  object
supervise_mode                        object
normal_class_fullqualified            object
loss                                  object
loss_empirical_cdf_clip_threshold    float64
validate/roc-auc-histavg             float64
validate/avg-precision-histavg       float64
scheduler_parameters                  object
normal_class                           int64
test/roc-auc                         float64
test/avg-precision                   float64
mvtec_class_type                      object
dtype: object


,Name,supervise_mode,normal_class_fullqualified,loss,loss_empirical_cdf_clip_threshold,validate/roc-auc-histavg,validate/avg-precision-histavg,scheduler_parameters,normal_class,test/roc-auc,test/avg-precision,mvtec_class_type
0,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.905855,0.341559,[0.9985],14,0.888598,0.319318,object
1,mvtec.mvtec.cls14.it01,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.893223,0.335284,[0.9985],14,0.863413,0.275698,object
2,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.923746,0.341422,[0.985],14,0.927588,0.342631,object
3,mvtec.mvtec.cls14.it01,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.925055,0.324699,[0.985],14,0.934807,0.332482,object
4,mvtec.mvtec.cls14.it00,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.875511,0.219284,[0.985],14,0.913435,0.248363,object


In [3]:
data_df = pd.read_csv(data_fpath) 
df = data_df
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: float(x.lstrip("[").rstrip("]")))
df["scheduler_parameters"] = df["scheduler_parameters"].apply(lambda x: {0.985000: "higher decay", 0.998500: "equiv. decay"}[x])
df.drop(columns=["Name"], inplace=True)
check_df(df)

(610, 11)
supervise_mode                        object
normal_class_fullqualified            object
loss                                  object
loss_empirical_cdf_clip_threshold    float64
validate/roc-auc-histavg             float64
validate/avg-precision-histavg       float64
scheduler_parameters                  object
normal_class                           int64
test/roc-auc                         float64
test/avg-precision                   float64
mvtec_class_type                      object
dtype: object


,supervise_mode,normal_class_fullqualified,loss,loss_empirical_cdf_clip_threshold,validate/roc-auc-histavg,validate/avg-precision-histavg,scheduler_parameters,normal_class,test/roc-auc,test/avg-precision,mvtec_class_type
0,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.905855,0.341559,equiv. decay,14,0.888598,0.319318,object
1,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.893223,0.335284,equiv. decay,14,0.863413,0.275698,object
2,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.923746,0.341422,higher decay,14,0.927588,0.342631,object
3,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.925055,0.324699,higher decay,14,0.934807,0.332482,object
4,synthetic-anomaly-confetti,mvtec_14_zipper,pixelwise-batch-avg,NaN,0.875511,0.219284,higher decay,14,0.913435,0.248363,object


In [4]:
data_df = df

In [5]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

def get_color_mask_compare_by_line_every_n_cols(pt, n):
    
    if n > 2:
        raise NotImplemented("n > 2 is not implemented ===> deal with color scale!")
    
    assert pt.shape[1] % n == 0, f"pt.shape[1] % n != 0: {pt.shape[1] % n}"

    def get_ordering_signal(arr: np.ndarray) -> np.ndarray:
        """at each line, the values are assigned a value from 0 to 1 respecting the ordering, such that 0 is the min and 1 is the max"""
        # it's ok to round up to 3 decimals because i will probably never have more than 1001 values
        # and it makes the table easier to read/debug
        assert arr.shape[1] < 1001, f"arr.shape[1] < 1001: {arr.shape[1]}"
        return np.round(np.argsort(arr) / (n - 1), decimals=3)

    return np.concatenate(
        [   
            get_ordering_signal(pt.iloc[:, i * n:(i + 1) *n].values)
            for i in range(pt.shape[1] // n)
        ],
        axis=1,
    )
    
COLOR_0 = 'background-color: #DC143C55'
COLOR_1 = 'background-color: #228B2255'
    
def color_from_mask(color_mask: np.ndarray):
    
    def color(_):
        colormap = np.empty_like(color_mask, dtype=object)
        colormap[color_mask == 0] = COLOR_0
        colormap[color_mask == 1] = COLOR_1
        return colormap
    
    return color

In [6]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from functools import partial

df = data_df.copy()

row_cols = ["mvtec_class_type", "normal_class_fullqualified"]
column_cols = ["supervise_mode", "loss", "scheduler_parameters"]

metrics = {
    # "test/avg-precision": ["mean", "std"],
    # "validate/avg-precision-histavg": ["mean",],
    "test/roc-auc": ["mean", "std"],
    "validate/roc-auc-histavg": ["mean",],
}
metric_cols = [(k, v) for k, list_vals in metrics.items() for v in list_vals]

pt_all_metrics = pd.pivot_table(df, index=row_cols, columns=column_cols, aggfunc=metrics)

# when it's a percentage
pt_all_metrics = 100 * pt_all_metrics

# pt_test_avg = pt_all_metrics[("test/avg-precision", "mean")]
pt_test_avg = pt_all_metrics[("test/roc-auc", "mean")]
pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
pt_test_avg.loc[("", "texture mean"), :] = pt_test_avg.loc["texture"].mean(axis=0)
pt_test_avg.loc[("", "all mean"), :] = pt_test_avg.mean(axis=0)

# pt_test_std = pt_all_metrics[("test/avg-precision", "std")]
pt_test_std = pt_all_metrics[("test/roc-auc", "std")]
pt_test_std.loc[("", "object mean"), :] = pt_test_std.loc["object"].mean(axis=0)
pt_test_std.loc[("", "texture mean"), :] = pt_test_std.loc["texture"].mean(axis=0)
pt_test_std.loc[("", "all mean"), :] = pt_test_std.mean(axis=0)

# pt_validate_histmean_avg = pt_all_metrics[("validate/avg-precision-histavg", "mean")]
pt_validate_histmean_avg = pt_all_metrics[("validate/roc-auc-histavg", "mean")]
pt_validate_histmean_avg.loc[("", "object mean"), :] = pt_validate_histmean_avg.loc["object"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "texture mean"), :] = pt_validate_histmean_avg.loc["texture"].mean(axis=0)
pt_validate_histmean_avg.loc[("", "all mean"), :] = pt_validate_histmean_avg.mean(axis=0)

pt_composed = pt_test_avg.applymap("{:.1f}".format) + " " + pt_test_std.applymap("({:.1f})".format) + " " + pt_validate_histmean_avg.applymap("[{:.1f}]".format)

/tmp/ipykernel_37656/1442041152.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_avg.loc[("", "object mean"), :] = pt_test_avg.loc["object"].mean(axis=0)
/tmp/ipykernel_37656/1442041152.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_test_std.loc[("", "object mean"), :] = pt_test_std.loc["object"].mean(axis=0)
/tmp/ipykernel_37656/1442041152.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pt_validate_histmean_a

# compare all

In [7]:
pt = pt_composed.copy()
pt_color_from = pt_test_avg.copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

# fixed loss

## old-fcdd

In [8]:
pt = pt_composed.loc[:, (slice(None), "old-fcdd")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), "old-fcdd")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

## pixelwise-batch-avg

In [9]:
pt = pt_composed.loc[:, (slice(None), "pixelwise-batch-avg")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), "pixelwise-batch-avg")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

# fixed decay

## higher decay

In [10]:
pt = pt_composed.loc[:, (slice(None), slice(None), "higher decay")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), slice(None), "higher decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

## equiv. decay

In [11]:
pt = pt_composed.loc[:, (slice(None), slice(None), "equiv. decay")].copy()
pt_color_from = pt_test_avg.loc[:, (slice(None), slice(None), "equiv. decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
pt.style.apply(color_from_mask(color_mask), axis=None)

In [12]:
pt = pt_composed.loc[:, (slice(None), "old-fcdd", "equiv. decay")].copy()
pt
pt_color_from = pt_test_avg.loc[:, (slice(None), "old-fcdd", "equiv. decay")].copy()
color_mask = get_color_mask_compare_by_line_every_n_cols(pt_color_from, n=2)
# pt.style.apply(color_from_mask(color_mask), axis=None)

supervise_mode                                    real-anomaly  \
loss                                                  old-fcdd   
scheduler_parameters                              equiv. decay   
mvtec_class_type normal_class_fullqualified                      
object           mvtec_00_bottle             96.4 (1.1) [92.9]   
                 mvtec_01_cable              94.5 (0.7) [91.8]   
                 mvtec_02_capsule            89.4 (1.5) [88.5]   
                 mvtec_05_hazelnut           97.1 (1.0) [96.1]   
                 mvtec_07_metal_nut          93.5 (2.2) [94.7]   
                 mvtec_08_pill               95.8 (2.5) [94.9]   
                 mvtec_09_screw              91.0 (1.4) [90.1]   
                 mvtec_11_toothbrush         86.9 (5.6) [84.1]   
                 mvtec_12_transistor         91.2 (2.3) [83.0]   
                 mvtec_14_zipper             98.3 (0.4) [93.8]   
texture          mvtec_03_carpet             98.8 (0.1) [98.3]   
                 mvtec_04_grid               93.3 (4.3) [86.3]   
                 mvtec_06_leather            98.9 (0.1) [98.5]   
                 mvtec_10_tile               98.7 (0.1) [98.4]   
                 mvtec_13_wood               95.0 (1.5) [93.0]   
                 object mean                 93.4 (1.9) [91.0]   
                 texture mean                97.0 (1.2) [94.9]   
                 all mean                    94.7 (1.6) [92.4]   

supervise_mode                              synthetic-anomaly-confetti  
loss                                                          old-fcdd  
scheduler_parameters                                      equiv. decay  
mvtec_class_type normal_class_fullqualified                             
object           mvtec_00_bottle                     87.8 (1.3) [85.0]  
                 mvtec_01_cable                      89.4 (1.1) [87.8]  
                 mvtec_02_capsule                    90.3 (1.1) [89.0]  
                 mvtec_05_hazelnut                   82.2 (1.2) [85.8]  
                 mvtec_07_metal_nut                  51.2 (2.7) [52.4]  
                 mvtec_08_pill                       30.4 (0.7) [33.8]  
                 mvtec_09_screw                      80.0 (4.2) [82.9]  
                 mvtec_11_toothbrush                 35.8 (9.0) [47.4]  
                 mvtec_12_transistor                 84.0 (1.0) [78.6]  
                 mvtec_14_zipper                     89.2 (2.0) [88.3]  
texture          mvtec_03_carpet                     93.2 (0.6) [93.9]  
                 mvtec_04_grid                       85.4 (1.6) [80.3]  
                 mvtec_06_leather                    97.8 (0.3) [97.4]  
                 mvtec_10_tile                       97.6 (0.2) [96.3]  
                 mvtec_13_wood                       90.9 (1.0) [89.6]  
                 object mean                         72.0 (2.4) [73.1]  
                 texture mean                        93.0 (0.7) [91.5]  
                 all mean                            79.4 (1.8) [79.6]